In [1]:
import numpy as np
import random
from scipy.spatial import distance

#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:

X = np.array(features)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)



In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]


In [11]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(100,), activation='logistic', max_iter=10, alpha=1e-4,
                    solver='adam', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

In [13]:
for Xtnn, ytnn in zip(X_train, y_train):
    X_train_pairs = []
    y_train_pair_lables = []
    dists = []
    dists_indexes = []
    for i,(Xtnn2, ytnn2) in enumerate (zip(X_train, y_train)):
        if (ytnn != ytnn2):
            dist = distance.euclidean(Xtnn, Xtnn2)
            dists.append(dist)
            dists_indexes.append(i)
    
    dists = np.array(dists)
    dists_indexes = np.array(dists_indexes)
    _indexes = np.argsort(dists)

    # Sorted distances and labels
    dists, dists_indexes = dists[_indexes], dists_indexes[_indexes] 
        
    for Xtnn2, ytnn2, camId_t2 in zip(X_train, y_train, camId_train):
        if (ytnn == ytnn2):
            for i in range (30):
                Xtnn3 = X_train[dists_indexes[i]]
                ytnn3 = y_train[dists_indexes[i]]                 

                dist2 = 0
                if(ytnn == ytnn3):
                    dist3 = 0
                else:
                    dist3 = 100

                Xconcat = np.concatenate((Xtnn,Xtnn2,Xtnn3), axis = None)
                X_train_pairs.append(Xconcat)
                y_train_pair_lables.append((dist2,dist3))   

                Xconcat = np.concatenate((Xtnn,Xtnn3,Xtnn2), axis = None)
                X_train_pairs.append(Xconcat)
                y_train_pair_lables.append((dist3,dist2)) 

    y_train_pair_lables = np.array(y_train_pair_lables)
    X_train_pairs = np.array(X_train_pairs)
    if (X_train_pairs.shape[0] != 0):
            mlp.partial_fit(X_train_pairs, y_train_pair_lables)      

Iteration 1, loss = 2146.34021590
Iteration 2, loss = 1531.78769166
Iteration 3, loss = 1279.88329814
Iteration 4, loss = 1276.94434117
Iteration 5, loss = 1348.17345527
Iteration 6, loss = 1361.08533351
Iteration 7, loss = 1314.03940797
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 8, loss = 1265.51775529
Iteration 9, loss = 1251.51232274
Iteration 10, loss = 1263.09835709
Iteration 11, loss = 1272.18331298
Iteration 12, loss = 1266.45904274
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 13, loss = 1255.22568232
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 14, loss = 1250.49227713
Iteration 15, loss = 1252.81148092
Iteration 16, loss = 1255.11803084
Iteration 17, loss = 1253.70207117
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 18, loss = 1251.05508989
Training loss d

Iteration 81, loss = 1250.29776191
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 82, loss = 1250.29926983
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 83, loss = 1250.30075991
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 84, loss = 1250.30218945
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 85, loss = 1250.30356596
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 86, loss = 1250.30492872
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 87, loss = 1250.30630973
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 88, loss = 1250.30771005
Training loss did not improve more than tol=0.000100 for two consecutive epochs. St

Iteration 148, loss = 1250.37222864
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 149, loss = 1250.37272040
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 150, loss = 1250.37387116
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 151, loss = 1250.37531695
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 152, loss = 1250.37649690
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 153, loss = 1250.37725489
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 154, loss = 1251.15498183
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 155, loss = 1250.71589821
Training loss did not improve more than tol=0.000100 for two consecutive ep

Iteration 215, loss = 1250.39141776
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 216, loss = 1250.42879523
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 217, loss = 1250.45038543
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 218, loss = 1250.43871096
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 219, loss = 1250.41544440
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 220, loss = 1250.41011132
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 221, loss = 1250.42489832
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 222, loss = 1250.43830946
Training loss did not improve more than tol=0.000100 for two consecutive ep

Iteration 280, loss = 1251.14637149
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 281, loss = 1250.68635258
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 282, loss = 1250.56351494
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 283, loss = 1250.71605175
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 284, loss = 1250.90507918
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 285, loss = 1250.33782892
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 286, loss = 1250.11511700
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 287, loss = 1250.15589017
Training loss did not improve more than tol=0.000100 for two consecutive ep

...

Iteration 7076, loss = 1250.97776971
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7077, loss = 1251.23469286
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7078, loss = 1251.56814107
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7079, loss = 1251.65041352
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7080, loss = 1251.23123957
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7081, loss = 1251.06238784
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7082, loss = 1251.17949866
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7083, loss = 1251.35536459
Training loss did not improve more than tol=0.000100 for two consec

Iteration 7141, loss = 1251.87312192
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7142, loss = 1251.84430433
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7143, loss = 1251.25411935
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7144, loss = 1251.22451594
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7145, loss = 1251.46331722
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7146, loss = 1251.62260656
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7147, loss = 1251.85126194
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7148, loss = 1259.39163596
Training loss did not improve more than tol=0.000100 for two consec

Iteration 7206, loss = 1250.90660539
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7207, loss = 1250.85152669
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7208, loss = 1250.83211139
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7209, loss = 1250.86272704
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7210, loss = 1250.89187566
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7211, loss = 1250.88691353
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7212, loss = 1250.86218788
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7213, loss = 1250.85083600
Training loss did not improve more than tol=0.000100 for two consec

Iteration 7271, loss = 1251.35551308
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7272, loss = 1251.19388382
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7273, loss = 1250.69280817
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7274, loss = 1250.49234999
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7275, loss = 1250.83189145
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7276, loss = 1250.65355826
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7277, loss = 1251.22877205
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7278, loss = 1251.07205733
Training loss did not improve more than tol=0.000100 for two consec

Iteration 7336, loss = 1250.80180343
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7337, loss = 1250.80093750
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7338, loss = 1251.15843140
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7339, loss = 1255.83849301
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7340, loss = 1275.68317441
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7341, loss = 1281.92474547
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7342, loss = 1265.35509982
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7343, loss = 1255.57830309
Training loss did not improve more than tol=0.000100 for two consec

In [14]:
def get_acc_score(y_valid, y_q, tot_label_occur):
    recall = 0
    true_positives = 0
    
    k = 0
    
    max_rank = 30
    
    rank_A = np.zeros(max_rank)
    AP_arr = np.zeros(11)
    
    while (recall < 1) or (k < max_rank):
        if (y_valid[k] == y_q):
            
            true_positives = true_positives + 1
            recall = true_positives/tot_label_occur
            precision = true_positives/(k+1)
            
            AP_arr[round((recall-0.05)*10)] = precision
            
            for n in range (k, max_rank):
                rank_A[n] = 1
            
        k = k+1
        
    max_precision = 0
    for i in range(10, -1, -1):
        max_precision = max(max_precision, AP_arr[i])
        AP_arr[i] = max_precision
    
    AP_ = AP_arr.sum()/11
    
    return AP_, rank_A

In [15]:
from scipy.spatial import distance

def evaluate_metric(X_query, camId_query, y_query, X_gallery, camId_gallery, y_gallery, metric = mlp, parameters = None):

    rank_accuracies = []
    AP = []

    # Break condition for testing
    #q = 0

    for query, camId_q, y_q in zip(X_query, camId_query, y_query):
        q_g_dists = []
        y_valid = []
        for gallery, camId_g, y_g  in zip(X_gallery, camId_gallery, y_gallery):
            if ((camId_q == camId_g) and (y_q == y_g)):
                continue
            else:
                if metric == mlp:
                    dist = mlp.predict(np.concatenate((query, gallery, np.zeros(2048))).reshape((1,6144)))[0][0]
                elif metric == 'minkowski':
                    dist = distance.minkowski(query, gallery, parameters)
                else:
                    raise NameError('Specified metric not supported')           
                q_g_dists.append(dist)
                y_valid.append(y_g)
    
        tot_label_occur = y_valid.count(y_q)
    
        q_g_dists = np.array(q_g_dists)
        y_valid = np.array(y_valid)
        
    
        _indexes = np.argsort(q_g_dists)
    
        # Sorted distances and labels
        q_g_dists, y_valid = q_g_dists[_indexes], y_valid[_indexes]
    
        AP_, rank_A = get_acc_score(y_valid, y_q, tot_label_occur)
    
        AP.append(AP_)
        
        rank_accuracies.append(rank_A) 
    
        #if q  > 5:
        #    break
        #q = q+1

    rank_accuracies = np.array(rank_accuracies)

    total = rank_accuracies.shape[0]
    rank_accuracies = rank_accuracies.sum(axis = 0)
    rank_accuracies = np.divide(rank_accuracies, total)

    i = 0
    print ('Accuracies by Rank:')
    while i < rank_accuracies.shape[0]:
        print('Rank ', i+1, ' = %.2f%%' % (rank_accuracies[i] * 100), '\t',
              'Rank ', i+2, ' = %.2f%%' % (rank_accuracies[i+1] * 100), '\t',
              'Rank ', i+3, ' = %.2f%%' % (rank_accuracies[i+2] * 100), '\t',
              'Rank ', i+4, ' = %.2f%%' % (rank_accuracies[i+3] * 100), '\t',
              'Rank ', i+5, ' = %.2f%%' % (rank_accuracies[i+4] * 100))
        i = i+5

    AP = np.array(AP)

    mAP = AP.sum()/AP.shape[0]
    print('mAP = %.2f%%' % (mAP * 100))
    
    return rank_accuracies, mAP

In [16]:
rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric = mlp,
                                       parameters = None)

Accuracies by Rank:
Rank  1  = 0.07% 	 Rank  2  = 0.14% 	 Rank  3  = 0.14% 	 Rank  4  = 0.14% 	 Rank  5  = 0.21%
Rank  6  = 0.29% 	 Rank  7  = 0.36% 	 Rank  8  = 0.36% 	 Rank  9  = 0.36% 	 Rank  10  = 0.36%
Rank  11  = 0.36% 	 Rank  12  = 0.43% 	 Rank  13  = 0.64% 	 Rank  14  = 0.64% 	 Rank  15  = 0.64%
Rank  16  = 0.64% 	 Rank  17  = 0.64% 	 Rank  18  = 0.79% 	 Rank  19  = 0.79% 	 Rank  20  = 0.79%
Rank  21  = 0.79% 	 Rank  22  = 0.86% 	 Rank  23  = 0.93% 	 Rank  24  = 0.93% 	 Rank  25  = 1.00%
Rank  26  = 1.07% 	 Rank  27  = 1.14% 	 Rank  28  = 1.21% 	 Rank  29  = 1.36% 	 Rank  30  = 1.36%
mAP = 0.35%
